In [ ]:
# ! pip install nba_api

In [1]:
import json
import time
import pandas as pd
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playergamelog, teamgamelog

```
{

    
}
```

In [2]:
_teams = teams.get_teams()
_players = players.get_active_players()

In [ ]:
result = []
FILE_NAME = 'player_data.json'

team_name_to_id = {}
for team in _teams:
    team_name_to_id[team['abbreviation']] = team['id']

for player in _players[:2]:
    print(f'GETTING PLAYER {player["full_name"]}:{player["id"]}')
    p_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id']).get_normalized_dict()

    player_info = p_info['CommonPlayerInfo'][0]
    
    if len(p_info['PlayerHeadlineStats']) == 0:
        continue

    player_headline_stats = p_info['PlayerHeadlineStats'][0]
    del player_headline_stats['PLAYER_ID']
    del player_headline_stats['PLAYER_NAME']
    del player_headline_stats['TimeFrame']

    player_gamelog = playergamelog.PlayerGameLog(player_id=player['id']).get_normalized_dict()['PlayerGameLog']
    for game in player_gamelog:
        game['TEAM_ABBREVIATION'] = game['MATCHUP'][:3]
        game['TEAM_ID'] = team_name_to_id[game['TEAM_ABBREVIATION']]

    result.append({ **player_info, 'CAREER_STATS': player_headline_stats, 'GAMELOG': player_gamelog })
    time.sleep(0.5)

with open(FILE_NAME, 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

In [ ]:
with open('strings.json') as f:
    d = json.load(f)
    print(d)

In [ ]:
df = pd.DataFrame()

for person in result:
    if len(person['GAMELOG']) == 0:
        continue
    df1 = pd.DataFrame(person, columns=['DISPLAY_FIRST_LAST', 'PERSON_ID', 'TEAM_ID'], index=[0])
    df2 = pd.DataFrame(person['GAMELOG'])
    merged = pd.merge(df1, df2, left_on='PERSON_ID', right_on='Player_ID', how='outer')
    df = pd.concat([df, merged])
    
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.columns

Find the player who scored the most points in an individual game last season.

Now do the same for rebounds, blocks, and assists.

Which player played the most minutes last season?

How many players played in at least 82 games last season?

Find the player who scored the most points for the entire season.

Now do the same for rebounds, blocks, and assists.

Find which team scored the most points for the entire season.

Now do the same for rebounds, blocks, and assists.

Which team had the most players play for them last season?

Did any players play for more than one team last year?

Which player(s) played for the most teams last season?

Find the player who scored the least amount of points while playing in at least 40 games last season.

Find the player who scored the least amount of points while playing in at least 40 games and averageing at least 15 minutes per game last season.

Which player scored the largest share of points for their team throughout the season.